# Dataset Flexível com MapBiomas para Análise de Uso da Terra

## Extração com dados de satélite e agregação flexível

Este notebook extrai dados da [Base dos Dados](https://basedosdados.org/) usando MapBiomas (dados de satélite) para identificar áreas dedicadas a diferentes culturas, com opção de agregação por município, microrregião ou AMC.

### Principais melhorias:
- ✅ **MapBiomas**: Dados de satélite mais precisos que autodeclaração
- ✅ **Agregação flexível**: Município, Microrregião ou AMC
- ✅ **Cobertura completa**: Todas as unidades geográficas incluídas
- ✅ **Múltiplas culturas**: Áreas de agricultura, cana, soja, café, cítricos e outras


## 0. Configuração e Setup

Configure aqui os parâmetros principais da extração:


In [17]:
# Importar bibliotecas
import basedosdados as bd
import pandas as pd
import numpy as np
import os
from datetime import datetime

# CONFIGURAÇÕES PRINCIPAIS
PROJECT_ID = "bdsdasd"
ANOS = list(range(2003, 2022))

# CONFIGURAÇÃO DE AGREGAÇÃO
# Dataset mantém granularidade municipal mas inclui id_microrregiao para flexibilidade
NIVEL_AGREGACAO = 'municipio'  # Mantém id_municipio como menor unidade

# CULTURAS COM MAIOR USO DE IRRIGAÇÃO (Top 5 no Brasil)
CULTURAS_IRRIGADAS = {
    'Cana-de-açúcar': 'cana',
    'Arroz': 'arroz', 
    'Soja': 'soja',
    'Milho': 'milho',
    'Feijão': 'feijao'
}

# Criar diretórios
os.makedirs("output", exist_ok=True)
os.makedirs("data", exist_ok=True)

print("✅ Configuração completa!")
print(f"📅 Período: {ANOS[0]}-{ANOS[-1]}")
print(f"🎯 Granularidade: MUNICIPAL (com id_microrregiao para agregação flexível)")
print(f"💧 Culturas irrigadas a extrair: {', '.join(CULTURAS_IRRIGADAS.keys())}")
print(f"🛰️ Categorias MapBiomas: Agricultura, Cana, Soja, Café, Cítricos, Outras Lavouras")


✅ Configuração completa!
📅 Período: 2003-2021
🎯 Granularidade: MUNICIPAL (com id_microrregiao para agregação flexível)
💧 Culturas irrigadas a extrair: Cana-de-açúcar, Arroz, Soja, Milho, Feijão
🛰️ Categorias MapBiomas: Agricultura, Cana, Soja, Café, Cítricos, Outras Lavouras


## 1. Mapeamentos Geográficos

Obter os mapeamentos necessários para agregação:


In [18]:
# PASSO 1.1: Obter diretório de municípios
query_municipios = """
SELECT 
    id_municipio,
    id_microrregiao,
    nome AS nome_municipio,
    sigla_uf
FROM 
    `basedosdados.br_bd_diretorios_brasil.municipio`
"""

print("🗺️ Baixando diretório de municípios...")
df_municipios = bd.read_sql(query_municipios, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_municipios):,} municípios mapeados")
print(f"✅ {df_municipios['id_microrregiao'].nunique()} microrregiões identificadas")

# PASSO 1.2: Obter mapeamento AMC se necessário
if NIVEL_AGREGACAO == 'amc':
    # Query AMC ajustada
    query_amc = f"""
    SELECT DISTINCT
        id_amc,
        id_municipio
    FROM 
        `basedosdados.br_ibge_amc.municipio_de_para`
    WHERE 
        ano_de <= {ANOS[0]} 
        AND ano_para >= {ANOS[-1]}
    """
    
    print("\n🗺️ Baixando mapeamento AMC...")
    df_amc = bd.read_sql(query_amc, billing_project_id=PROJECT_ID)
    
    # Adicionar AMC ao dataframe de municípios
    df_municipios = df_municipios.merge(df_amc, on='id_municipio', how='left')
    print(f"✅ {df_amc['id_amc'].nunique()} AMCs identificadas")


🗺️ Baixando diretório de municípios...
Downloading: 100%|██████████|
✅ 5,570 municípios mapeados
✅ 558 microrregiões identificadas


## 2. Dados MapBiomas (Uso da Terra)

Extrair dados de cobertura da terra por satélite:


In [ ]:
# PASSO 2: Baixar dados MapBiomas
# IDs das categorias de interesse
CATEGORIAS_MAPBIOMAS = {
    '18': 'agricultura',
    '19': 'lavoura_temporaria',
    '39': 'soja',
    '20': 'cana',
    '41': 'outras_lavouras_temporarias',
    '46': 'cafe',
    '47': 'citricos'
}

ids_categorias = list(CATEGORIAS_MAPBIOMAS.keys())
ids_str = ', '.join([f"'{id}'" for id in ids_categorias])

query_mapbiomas = f"""
SELECT
    ano,
    id_municipio,
    id_classe,
    area
FROM 
    `basedosdados.br_mapbiomas_estatisticas.cobertura_municipio_classe`
WHERE 
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
    AND id_classe IN ({ids_str})
"""

print(f"🛰️ Baixando dados MapBiomas para {len(CATEGORIAS_MAPBIOMAS)} categorias...")
print(f"   IDs solicitados: {ids_categorias}")
df_mapbiomas_mun = bd.read_sql(query_mapbiomas, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_mapbiomas_mun):,} registros de cobertura")

# Verificar quais classes foram realmente baixadas
print("\n🔍 Classes únicas encontradas nos dados:")
classes_baixadas = df_mapbiomas_mun['id_classe'].unique()
print(f"   Classes baixadas: {sorted(classes_baixadas)}")
print(f"   Total de classes diferentes: {len(classes_baixadas)}")

# Pivotar para ter uma coluna por categoria
# Converter id_classe para string se necessário
df_mapbiomas_mun['id_classe_str'] = df_mapbiomas_mun['id_classe'].astype(str)
df_mapbiomas_mun['categoria'] = df_mapbiomas_mun['id_classe_str'].map(CATEGORIAS_MAPBIOMAS)

# Verificar se todas as categorias foram mapeadas
print("\n📊 Verificando mapeamento das categorias:")
print(f"   Categorias esperadas: {list(CATEGORIAS_MAPBIOMAS.values())}")
print(f"   Categorias encontradas: {df_mapbiomas_mun['categoria'].unique().tolist()}")
print(f"   Classes não mapeadas: {df_mapbiomas_mun[df_mapbiomas_mun['categoria'].isna()]['id_classe'].unique().tolist()}")

df_mapbiomas_pivot = df_mapbiomas_mun.pivot_table(
    index=['ano', 'id_municipio'],
    columns='categoria',
    values='area',
    fill_value=0
).reset_index()

# Adicionar prefixo às colunas
cols_mapbiomas = [col for col in df_mapbiomas_pivot.columns if col not in ['ano', 'id_municipio']]
for col in cols_mapbiomas:
    df_mapbiomas_pivot[f'area_{col}'] = df_mapbiomas_pivot[col]
    df_mapbiomas_pivot.drop(col, axis=1, inplace=True)

print(f"\n📋 Colunas criadas no MapBiomas:")
print(f"   {[col for col in df_mapbiomas_pivot.columns if col.startswith('area_')]}")

print(f"✅ Dados pivotados: {len(df_mapbiomas_pivot):,} registros município-ano")

# PASSO 2.1: Calcular área total dos municípios a partir do MapBiomas
# A área total é a soma de todas as classes de cobertura
print("\n📏 Calculando área total dos municípios...")

query_area_total = f"""
SELECT
    ano,
    id_municipio,
    SUM(area) as area_total_ha
FROM 
    `basedosdados.br_mapbiomas_estatisticas.cobertura_municipio_classe`
WHERE 
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
GROUP BY 
    ano, id_municipio
"""

df_area_total = bd.read_sql(query_area_total, billing_project_id=PROJECT_ID)

print(f"✅ Área total calculada para {df_area_total['id_municipio'].nunique():,} municípios")
print(f"   Registros município-ano: {len(df_area_total):,}")

# Adicionar área total ao dataframe pivotado
df_mapbiomas_pivot = df_mapbiomas_pivot.merge(df_area_total, on=['ano', 'id_municipio'], how='left')


🛰️ Baixando dados MapBiomas para 7 categorias...
   IDs solicitados: ['18', '19', '39', '20', '41', '46', '47']
Downloading: 100%|██████████|
✅ 214,548 registros de cobertura

🔍 Classes únicas encontradas nos dados:
   Classes baixadas: ['20', '39', '41', '46', '47']
   Total de classes diferentes: 5

📊 Verificando mapeamento das categorias:
   Categorias esperadas: ['agricultura', 'lavoura_temporaria', 'soja', 'cana', 'outras_lavouras_temporarias', 'cafe', 'citricos']
   Categorias encontradas: ['cana', 'soja', 'outras_lavouras_temporarias', 'cafe', 'citricos']
   Classes não mapeadas: []

📋 Colunas criadas no MapBiomas:
   ['area_cafe', 'area_cana', 'area_citricos', 'area_outras_lavouras_temporarias', 'area_soja']
✅ Dados pivotados: 94,737 registros município-ano

📏 Calculando área total dos municípios...
Downloading: 100%|██████████|
✅ Área total calculada para 5,566 municípios
   Registros município-ano: 105,754


## 3. Estações Meteorológicas

Identificar tratamento por estações automáticas:


In [20]:
# PASSO 3: Estações meteorológicas
query_estacoes = """
SELECT
    e.id_municipio,
    e.id_estacao,
    e.estacao AS nome_estacao,
    EXTRACT(YEAR FROM e.data_fundacao) AS ano_fundacao
FROM
    `basedosdados.br_inmet_bdmep.estacao` e
WHERE
    e.data_fundacao IS NOT NULL
    AND e.id_municipio IS NOT NULL
"""

print("🌡️ Baixando dados de estações...")
df_estacoes_mun = bd.read_sql(query_estacoes, billing_project_id=PROJECT_ID)

# Adicionar informação geográfica
df_estacoes_full = df_estacoes_mun.merge(
    df_municipios, 
    on='id_municipio', 
    how='left'
)

print(f"✅ {df_estacoes_full['id_estacao'].nunique()} estações identificadas")


🌡️ Baixando dados de estações...
Downloading: 100%|██████████|
✅ 610 estações identificadas


## 4. População e PIB

Dados socioeconômicos:


In [21]:
# PASSO 4.1: População
query_pop = f"""
SELECT
    ano,
    id_municipio,
    populacao
FROM
    `basedosdados.br_ibge_populacao.municipio`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
"""

print("👥 Baixando dados de população...")
df_pop_mun = bd.read_sql(query_pop, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pop_mun):,} registros")

# PASSO 4.2: PIB
query_pib = f"""
SELECT
    ano,
    id_municipio,
    pib,
    va_agropecuaria
FROM
    `basedosdados.br_ibge_pib.municipio`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
"""

print("\n💰 Baixando dados de PIB...")
df_pib_mun = bd.read_sql(query_pib, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pib_mun):,} registros")


👥 Baixando dados de população...
Downloading: 100%|██████████|
✅ 105,776 registros

💰 Baixando dados de PIB...
Downloading: 100%|██████████|
✅ 105,830 registros


## 5. Produção Agrícola (PAM)

Dados de produção para as 5 culturas com maior uso de irrigação:


In [22]:
# PASSO 5: Dados PAM para as 5 culturas irrigadas
# Primeiro verificar os nomes exatos dos produtos disponíveis
query_produtos = f"""
SELECT DISTINCT
    produto
FROM
    `basedosdados.br_ibge_pam.lavoura_temporaria`
WHERE
    ano = {ANOS[0]}
    AND produto IN ('Cana-de-açúcar', 'Arroz', 'Arroz (em casca)', 'Soja', 'Soja (em grão)', 
                    'Milho', 'Milho (em grão)', 'Feijão', 'Feijão (em grão)')
ORDER BY produto
"""

print("🔍 Verificando nomes de produtos disponíveis...")
df_produtos = bd.read_sql(query_produtos, billing_project_id=PROJECT_ID)
print(f"   Produtos encontrados: {df_produtos['produto'].tolist()}")

# Ajustar nomes dos produtos baseado no que foi encontrado
CULTURAS_IRRIGADAS_CORRIGIDO = {
    'Cana-de-açúcar': 'cana',
    'Arroz (em casca)': 'arroz',  # Nome correto no PAM
    'Soja (em grão)': 'soja',     # Nome correto no PAM
    'Milho (em grão)': 'milho',   # Nome correto no PAM
    'Feijão (em grão)': 'feijao'  # Nome correto no PAM
}

# Criar lista de produtos para query
produtos_list = list(CULTURAS_IRRIGADAS_CORRIGIDO.keys())
produtos_str = "', '".join(produtos_list)

query_pam = f"""
SELECT
    ano,
    id_municipio,
    produto,
    area_plantada,
    area_colhida,
    quantidade_produzida,
    rendimento_medio_producao AS produtividade,
    valor_producao
FROM
    `basedosdados.br_ibge_pam.lavoura_temporaria`
WHERE
    ano BETWEEN {ANOS[0]} AND {ANOS[-1]}
    AND produto IN ('{produtos_str}')
"""

print(f"\n🌾 Baixando dados PAM para {len(CULTURAS_IRRIGADAS_CORRIGIDO)} culturas irrigadas...")
df_pam_raw = bd.read_sql(query_pam, billing_project_id=PROJECT_ID)
print(f"✅ {len(df_pam_raw):,} registros de produção")

# Criar identificador simplificado para cada cultura
df_pam_raw['cultura_id'] = df_pam_raw['produto'].map(CULTURAS_IRRIGADAS_CORRIGIDO)

# Pivotar para ter colunas separadas por cultura
df_pam_list = []
for cultura, cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.items():
    df_cultura = df_pam_raw[df_pam_raw['produto'] == cultura].copy()
    
    # Renomear colunas com prefixo da cultura
    df_cultura = df_cultura.rename(columns={
        'area_plantada': f'area_plantada_{cultura_id}',
        'area_colhida': f'area_colhida_{cultura_id}',
        'quantidade_produzida': f'quantidade_{cultura_id}',
        'produtividade': f'produtividade_{cultura_id}',
        'valor_producao': f'valor_{cultura_id}'
    })
    
    # Manter apenas colunas relevantes
    cols_keep = ['ano', 'id_municipio'] + [col for col in df_cultura.columns if cultura_id in col]
    df_cultura = df_cultura[cols_keep]
    
    df_pam_list.append(df_cultura)
    print(f"   - {cultura}: {len(df_cultura):,} registros")

# Fazer merge de todas as culturas
df_pam_mun = df_pam_list[0]
for df in df_pam_list[1:]:
    df_pam_mun = df_pam_mun.merge(df, on=['ano', 'id_municipio'], how='outer')

print(f"\n✅ Dataset PAM consolidado: {len(df_pam_mun):,} registros município-ano")


🔍 Verificando nomes de produtos disponíveis...
Downloading: 100%|██████████|
   Produtos encontrados: ['Arroz (em casca)', 'Cana-de-açúcar', 'Feijão (em grão)', 'Milho (em grão)', 'Soja (em grão)']

🌾 Baixando dados PAM para 5 culturas irrigadas...
Downloading: 100%|██████████|
✅ 528,485 registros de produção
   - Cana-de-açúcar: 105,697 registros
   - Arroz (em casca): 105,697 registros
   - Soja (em grão): 105,697 registros
   - Milho (em grão): 105,697 registros
   - Feijão (em grão): 105,697 registros

✅ Dataset PAM consolidado: 105,697 registros município-ano


## 6. Consolidação dos Dados

Consolidar todos os dados mantendo granularidade municipal:


In [23]:
# Manter dados municipais com informação de microrregião
print(f"🎯 Mantendo granularidade municipal com id_microrregiao")

# Base com todos os municípios e suas microrregiões
df_base = df_municipios[['id_municipio', 'id_microrregiao', 'sigla_uf']].copy()

# Criar painel completo município-ano
from itertools import product

all_municipios = df_base['id_municipio'].unique()

# Criar painel completo
painel = pd.DataFrame(
    list(product(all_municipios, ANOS)), 
    columns=['id_municipio', 'ano']
)

# Adicionar informação de microrregião e UF
painel = painel.merge(df_base, on='id_municipio', how='left')

print(f"✅ Painel criado: {len(painel):,} observações ({len(all_municipios)} municípios × {len(ANOS)} anos)")
print(f"✅ Microrregiões incluídas: {painel['id_microrregiao'].nunique()}")


🎯 Mantendo granularidade municipal com id_microrregiao
✅ Painel criado: 105,830 observações (5570 municípios × 19 anos)
✅ Microrregiões incluídas: 558


In [24]:
# ADICIONAR MAPBIOMAS
print("\n🛰️ Adicionando dados MapBiomas...")

# Colunas de área
area_cols = [col for col in df_mapbiomas_pivot.columns if col.startswith('area_')]

# Merge direto com painel
painel = painel.merge(df_mapbiomas_pivot, on=['ano', 'id_municipio'], how='left')

# Preencher zeros onde não há dados
for col in area_cols:
    painel[col] = painel[col].fillna(0)

print(f"✅ MapBiomas adicionado: {len(area_cols)} categorias de uso da terra")

# Listar todas as colunas de área disponíveis
print("\n📋 Colunas de área disponíveis no painel:")
area_cols_painel = [col for col in painel.columns if col.startswith('area_')]
print(f"   {area_cols_painel}")

# ADICIONAR ÁREA TOTAL E CALCULAR INDICADORES DE URBANIZAÇÃO
print("\n🏙️ Calculando indicadores de urbanização...")

# A área total já está no dataframe após o merge com MapBiomas
# Verificar se as colunas necessárias existem antes de calcular
if 'area_agropecuaria' in painel.columns and 'area_area_urbana' in painel.columns:
    # Calcular porcentagens usando agropecuária total (classe 14) e área urbana (classe 24)
    painel['pct_area_agropecuaria'] = (painel['area_agropecuaria'] / painel['area_total_ha'] * 100).round(2)
    painel['pct_area_urbana'] = (painel['area_area_urbana'] / painel['area_total_ha'] * 100).round(2)
else:
    print("⚠️ AVISO: Colunas area_agropecuaria ou area_area_urbana não encontradas!")
    print(f"   Colunas disponíveis: {[col for col in painel.columns if 'agropecuaria' in col or 'urbana' in col]}")
    
    # Criar colunas vazias temporariamente
    painel['pct_area_agropecuaria'] = 0
    painel['pct_area_urbana'] = 0

# Calcular razão entre área agropecuária e urbana
if 'area_agropecuaria' in painel.columns and 'area_area_urbana' in painel.columns:
    painel['razao_agropecuaria_urbana'] = np.where(
        painel['area_area_urbana'] > 0,
        painel['area_agropecuaria'] / painel['area_area_urbana'],
        np.inf  # Se não há área urbana, a razão é infinita
    )
else:
    painel['razao_agropecuaria_urbana'] = np.inf

# Criar variável categórica de urbanização
# Município é considerado urbano se:
# 1) % área urbana > 40% OU 
# 2) razão agropecuária/urbana < 1 (ou seja, área urbana > área agropecuária)
painel['municipio_urbano'] = (
    (painel['pct_area_urbana'] > 40) | 
    (painel['razao_agropecuaria_urbana'] < 1)
).astype(int)

# Estatísticas de urbanização
print(f"✅ Indicadores de urbanização calculados")
print(f"   - Municípios urbanos: {painel[painel['municipio_urbano'] == 1]['id_municipio'].nunique():,}")
print(f"   - Municípios rurais: {painel[painel['municipio_urbano'] == 0]['id_municipio'].nunique():,}")
print(f"   - % área urbana médio: {painel['pct_area_urbana'].mean():.1f}%")
print(f"   - % área agropecuária médio: {painel['pct_area_agropecuaria'].mean():.1f}%")



🛰️ Adicionando dados MapBiomas...
✅ MapBiomas adicionado: 6 categorias de uso da terra

📋 Colunas de área disponíveis no painel:
   ['area_cafe', 'area_cana', 'area_citricos', 'area_outras_lavouras_temporarias', 'area_soja', 'area_total_ha']

🏙️ Calculando indicadores de urbanização...
⚠️ AVISO: Colunas area_agropecuaria ou area_area_urbana não encontradas!
   Colunas disponíveis: []
✅ Indicadores de urbanização calculados
   - Municípios urbanos: 0
   - Municípios rurais: 5,570
   - % área urbana médio: 0.0%
   - % área agropecuária médio: 0.0%


In [25]:
# ADICIONAR ESTAÇÕES
print("\n🌡️ Adicionando dados de estações meteorológicas...")

# Agregar estações por município
df_estacoes_agg = df_estacoes_full.groupby('id_municipio').agg({
    'ano_fundacao': 'min',
    'id_estacao': 'count'
}).reset_index()
df_estacoes_agg.columns = ['id_municipio', 'primeiro_ano_estacao', 'num_estacoes']

# Adicionar ao painel
painel = painel.merge(df_estacoes_agg[['id_municipio', 'primeiro_ano_estacao']], on='id_municipio', how='left')

# Criar variáveis de tratamento
painel['primeiro_ano_tratamento'] = painel['primeiro_ano_estacao'].fillna(0).astype(int)
painel['tratado'] = (painel['primeiro_ano_tratamento'] != 0).astype(int)
painel['pos_tratamento'] = (
    (painel['ano'] >= painel['primeiro_ano_tratamento']) & 
    (painel['tratado'] == 1)
).astype(int)

print(f"✅ Municípios tratados: {painel[painel['tratado'] == 1]['id_municipio'].nunique()}")
print(f"✅ Municípios controle: {painel[painel['tratado'] == 0]['id_municipio'].nunique()}")



🌡️ Adicionando dados de estações meteorológicas...
✅ Municípios tratados: 587
✅ Municípios controle: 4983


In [ ]:
# ADICIONAR POPULAÇÃO E PIB
print("\n💰 Adicionando dados socioeconômicos...")

# Renomear colunas de população
df_pop_mun_renamed = df_pop_mun.rename(columns={'populacao': 'populacao_total'})

# Renomear colunas de PIB
df_pib_mun_renamed = df_pib_mun.rename(columns={
    'pib': 'pib_total',
    'va_agropecuaria': 'pib_agropecuario'
})

# Adicionar ao painel
painel = painel.merge(df_pop_mun_renamed, on=['ano', 'id_municipio'], how='left')
painel = painel.merge(df_pib_mun_renamed, on=['ano', 'id_municipio'], how='left')

# Calcular PIB per capita
# Usar .fillna(0) para tratar NAs na comparação
mask_pop_valida = painel['populacao_total'].fillna(0) > 0
painel['pib_per_capita'] = np.where(
    mask_pop_valida,
    painel['pib_total'] / painel['populacao_total'],
    np.nan
)

print(f"✅ Dados socioeconômicos adicionados")



💰 Adicionando dados socioeconômicos...


TypeError: boolean value of NA is ambiguous

In [29]:
# ADICIONAR PAM
print("\n🌾 Adicionando dados PAM das culturas irrigadas...")

# Adicionar ao painel
painel = painel.merge(df_pam_mun, on=['ano', 'id_municipio'], how='left')

# Preencher zeros onde não há produção
pam_cols = [col for col in painel.columns if any(cultura in col for cultura in CULTURAS_IRRIGADAS_CORRIGIDO.values())]
for col in pam_cols:
    painel[col] = painel[col].fillna(0)

# Garantir que produtividade seja 0 onde não há área colhida
for cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.values():
    col_area = f'area_colhida_{cultura_id}'
    col_prod = f'produtividade_{cultura_id}'
    if col_area in painel.columns and col_prod in painel.columns:
        painel[col_prod] = np.where(painel[col_area] == 0, 0, painel[col_prod])

print(f"✅ Dados PAM adicionados para {len(CULTURAS_IRRIGADAS_CORRIGIDO)} culturas irrigadas")
print(f"   Colunas de produtividade criadas:")
for cultura, cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.items():
    print(f"   - produtividade_{cultura_id}: {cultura}")



🌾 Adicionando dados PAM das culturas irrigadas...
✅ Dados PAM adicionados para 5 culturas irrigadas
   Colunas de produtividade criadas:
   - produtividade_cana: Cana-de-açúcar
   - produtividade_arroz: Arroz (em casca)
   - produtividade_soja: Soja (em grão)
   - produtividade_milho: Milho (em grão)
   - produtividade_feijao: Feijão (em grão)


In [30]:
# Remover observações com dados faltantes críticos
print(f"\n🧹 Removendo observações com dados faltantes...")
print(f"   - Antes: {len(painel):,} observações")

# Remover onde não temos população ou PIB
painel_final = painel.dropna(subset=['populacao_total', 'pib_total'])

print(f"   - Após remover NAs (população/PIB): {len(painel_final):,} observações")
print(f"   - Removidas: {len(painel) - len(painel_final):,} observações ({(len(painel) - len(painel_final))/len(painel)*100:.1f}%)")



🧹 Removendo observações com dados faltantes...
   - Antes: 105,830 observações
   - Após remover NAs (população/PIB): 105,764 observações
   - Removidas: 66 observações (0.1%)


In [ ]:
# Estatísticas do dataset final
print("\n📊 ESTATÍSTICAS DO DATASET FINAL")
print("=" * 50)
print(f"Granularidade: MUNICIPAL (com id_microrregiao)")
print(f"Período: {ANOS[0]}-{ANOS[-1]}")
print(f"Culturas irrigadas incluídas: {', '.join(CULTURAS_IRRIGADAS_CORRIGIDO.keys())}")
print(f"\nDimensões: {len(painel_final):,} observações × {len(painel_final.columns)} variáveis")
print(f"Municípios: {painel_final['id_municipio'].nunique():,}")
print(f"Microrregiões: {painel_final['id_microrregiao'].nunique():,}")
print(f"Estados: {painel_final['sigla_uf'].nunique()}")

print(f"\n🌡️ TRATAMENTO (Estações Meteorológicas):")
print(f"   - Municípios tratados: {painel_final[painel_final['tratado'] == 1]['id_municipio'].nunique():,}")
print(f"   - Municípios controle: {painel_final[painel_final['tratado'] == 0]['id_municipio'].nunique():,}")
print(f"   - Observações pós-tratamento: {painel_final['pos_tratamento'].sum():,}")

print(f"\n🏙️ URBANIZAÇÃO:")
print(f"   - Municípios urbanos: {painel_final[painel_final['municipio_urbano'] == 1]['id_municipio'].nunique():,} ({painel_final[painel_final['municipio_urbano'] == 1]['id_municipio'].nunique() / painel_final['id_municipio'].nunique() * 100:.1f}%)")
print(f"   - Municípios rurais: {painel_final[painel_final['municipio_urbano'] == 0]['id_municipio'].nunique():,} ({painel_final[painel_final['municipio_urbano'] == 0]['id_municipio'].nunique() / painel_final['id_municipio'].nunique() * 100:.1f}%)")
print(f"   - % área urbana médio: {painel_final['pct_area_urbana'].mean():.1f}%")
print(f"   - % área agropecuária médio: {painel_final['pct_area_agropecuaria'].mean():.1f}%")

print(f"\n🛰️ USO DA TERRA (MapBiomas - hectares):")
for col in area_cols:
    area_total = painel_final[col].sum()
    if area_total > 0:
        nome_categoria = col.replace('area_', '').replace('_', ' ').title()
        print(f"   - {nome_categoria}: {area_total:,.0f} ha total")

print(f"\n💧 PRODUÇÃO DAS CULTURAS IRRIGADAS (PAM):")
for cultura, cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.items():
    area_col = f'area_plantada_{cultura_id}'
    prod_col = f'quantidade_{cultura_id}'
    produtividade_col = f'produtividade_{cultura_id}'
    
    if area_col in painel_final.columns:
        produtores = (painel_final[area_col] > 0).sum()
        area_total = painel_final[area_col].sum()
        producao_total = painel_final[prod_col].sum()
        
        # Produtividade média apenas onde há produção
        mask_prod = painel_final[produtividade_col] > 0
        produtividade_media = painel_final[mask_prod][produtividade_col].mean() if mask_prod.any() else 0
        
        print(f"\n   📌 {cultura}:")
        print(f"      - Municípios produtores: {produtores:,}")
        print(f"      - Área plantada: {area_total:,.0f} ha")
        print(f"      - Produção: {producao_total:,.0f} ton")
        print(f"      - Produtividade média: {produtividade_media:,.1f} kg/ha")

print(f"\n💰 SOCIOECONÔMICO:")
print(f"   - População total: {painel_final.groupby('ano')['populacao_total'].sum().iloc[-1]:,.0f} (último ano)")
print(f"   - PIB per capita médio: R$ {painel_final['pib_per_capita'].mean():,.2f}")
print(f"   - Participação agropecuária no PIB: {(painel_final['pib_agropecuario'].sum() / painel_final['pib_total'].sum() * 100):.1f}%")



📊 ESTATÍSTICAS DO DATASET FINAL
Granularidade: MUNICIPAL (com id_microrregiao)
Período: 2003-2021
Culturas irrigadas incluídas: Cana-de-açúcar, Arroz (em casca), Soja (em grão), Milho (em grão), Feijão (em grão)

Dimensões: 105,764 observações × 52 variáveis
Municípios: 5,570
Microrregiões: 558
Estados: 27

🌡️ TRATAMENTO (Estações Meteorológicas):
   - Municípios tratados: 587
   - Municípios controle: 4,983
   - Observações pós-tratamento: 7,601

🏙️ URBANIZAÇÃO:
   - Municípios urbanos: 0 (0.0%)
   - Municípios rurais: 5,570 (100.0%)
   - % área urbana médio: 0.0%
   - % área agropecuária médio: 0.0%

🛰️ USO DA TERRA (MapBiomas - hectares):
   - Cafe: 192,696 ha total
   - Cana: 1,386,170 ha total
   - Citricos: 47,513 ha total
   - Outras Lavouras Temporarias: 1,664,227 ha total
   - Soja: 6,092,749 ha total
   - Total Ha: 135,134,243 ha total

💧 PRODUÇÃO DAS CULTURAS IRRIGADAS (PAM):

   📌 Cana-de-açúcar:
      - Municípios produtores: 66,253
      - Área plantada: 167,418,921 ha
 

KeyError: 'pib_per_capita'

## 8. Exportar Dataset Final


In [ ]:
# Organizar colunas na ordem desejada
cols_base = ['ano', 'id_municipio', 'id_microrregiao', 'sigla_uf', 'primeiro_ano_tratamento', 'tratado', 'pos_tratamento']
cols_socio = ['populacao_total', 'pib_total', 'pib_per_capita', 'pib_agropecuario']
cols_urbanizacao = ['area_total_ha', 'pct_area_agropecuaria', 'pct_area_urbana', 'razao_agropecuaria_urbana', 'municipio_urbano']
cols_mapbiomas = [col for col in painel_final.columns if col.startswith('area_') and col not in ['area_total_ha']]

# Colunas PAM organizadas por cultura
cols_pam = []
for cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.values():
    # Ordem: área plantada, área colhida, quantidade, produtividade, valor
    for prefixo in ['area_plantada', 'area_colhida', 'quantidade', 'produtividade', 'valor']:
        col = f'{prefixo}_{cultura_id}'
        if col in painel_final.columns:
            cols_pam.append(col)

# Remover coluna temporária se existir
if 'primeiro_ano_estacao' in painel_final.columns:
    painel_final = painel_final.drop('primeiro_ano_estacao', axis=1)

# Ordenar colunas
cols_order = cols_base + cols_socio + cols_urbanizacao + cols_pam + cols_mapbiomas
cols_final = [col for col in cols_order if col in painel_final.columns]
df_export = painel_final[cols_final].sort_values(['id_municipio', 'ano'])

# Nome do arquivo
timestamp = datetime.now().strftime('%Y%m%d')
filename = f'dataset_mapbiomas_municipio_culturas_irrigadas_{ANOS[0]}-{ANOS[-1]}_{timestamp}.csv'
filepath = f'data/{filename}'

# Salvar
df_export.to_csv(filepath, index=False)

print(f"\n✅ DATASET EXPORTADO COM SUCESSO!")
print(f"📁 Arquivo: {filepath}")
print(f"📏 Tamanho: {os.path.getsize(filepath) / 1024 / 1024:.1f} MB")
print(f"📊 Dimensões: {len(df_export):,} × {len(df_export.columns)}")

# Mostrar amostra
print(f"\n🔍 Amostra dos dados:")
df_export.head(10)



✅ DATASET EXPORTADO COM SUCESSO!
📁 Arquivo: data/dataset_mapbiomas_microrregiao_Cana-de-açúcar_2003-2021_20250912.csv
📏 Tamanho: 1.6 MB
📊 Dimensões: 10,602 × 20

🔍 Amostra dos dados:


,ano,id_microrregiao,sigla_uf,primeiro_ano_tratamento,tratado,pos_tratamento,pam_area_plantada,pam_area_colhida,pam_quantidade_produzida,pam_valor_producao,pam_produtividade,populacao_total,pib_total,pib_per_capita,pib_agropecuario,area_cafe,area_cana,area_citricos,area_outras_lavouras_temporarias,area_soja
0,2003,11001,RO,2007,1,0,82,82,5695.0,226.0,69.451220,448419,3173358000,7076.769718,170001000,0.0,0.0,0.0,2.514962,0.009672
1,2004,11001,RO,2007,1,0,99,99,6817.0,291.0,68.858586,497775,3885396000,7805.526593,175586000,0.0,0.0,0.0,4.568636,0.000000
2,2005,11001,RO,2007,1,0,92,92,6111.0,271.0,66.423913,485002,4385122000,9041.451375,195425000,0.0,0.0,0.0,7.502846,0.791378
3,2006,11001,RO,2007,1,0,117,117,7645.0,440.0,65.341880,497936,4723308000,9485.773272,192598000,0.0,0.0,0.0,10.881466,0.806394
4,2007,11001,RO,2007,1,1,75,75,4657.0,318.0,62.093333,474532,5169521000,10893.935499,230175000,0.0,0.0,0.0,12.199082,0.086411
5,2008,11001,RO,2007,1,1,78,78,4576.0,327.0,58.666667,486884,6118513000,12566.675019,340318000,0.0,0.0,0.0,13.045108,0.036946
6,2009,11001,RO,2007,1,1,104,104,5827.0,233.0,56.028846,493445,7786946000,15780.778000,367499000,0.0,0.0,0.0,18.187689,0.000000
7,2010,11001,RO,2007,1,1,73,73,4088.0,163.0,56.000000,540320,10298189000,19059.425896,426759000,0.0,0.0,0.0,20.663352,0.183904
8,2011,11001,RO,2007,1,1,47,47,2587.0,760.0,55.042553,550089,12509934000,22741.654532,508095000,0.0,0.0,0.0,22.798335,2.427921
9,2012,11001,RO,2007,1,1,0,0,0.0,0.0,0.000000,559543,13425222000,23993.190872,633448000,0.0,0.0,0.0,22.288220,3.582765


In [ ]:
# Salvar metadados
metadata = {
    'granularidade': 'municipal',
    'culturas_irrigadas': CULTURAS_IRRIGADAS_CORRIGIDO,
    'anos': f"{ANOS[0]}-{ANOS[-1]}",
    'n_observacoes': len(df_export),
    'n_municipios': df_export['id_municipio'].nunique(),
    'n_microrregions': df_export['id_microrregiao'].nunique(),
    'n_tratadas': df_export[df_export['tratado'] == 1]['id_municipio'].nunique(),
    'n_controle': df_export[df_export['tratado'] == 0]['id_municipio'].nunique(),
    'categorias_mapbiomas': list(CATEGORIAS_MAPBIOMAS.values()),
    'data_extracao': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

# Salvar metadados em JSON
import json
metadata_file = filepath.replace('.csv', '_metadata.json')
with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"\n📝 Metadados salvos: {metadata_file}")

# Salvar mapeamento geográfico
mapping_file = 'output/mapeamento_municipio_microrregiao.csv'
df_municipios.to_csv(mapping_file, index=False)
print(f"🗺️ Mapeamento salvo: {mapping_file}")


## 9. Dicionário de Variáveis

Criar documentação detalhada de todas as variáveis do dataset:


In [ ]:
# Criar dicionário de variáveis do dataset
descricao_variaveis_list = [
    # Variáveis de identificação
    {'variavel': 'ano', 
     'descricao': 'Ano da observação', 
     'tipo': 'int', 
     'unidade': 'ano', 
     'fonte': 'Base dos Dados'},
    
    {'variavel': 'id_municipio', 
     'descricao': 'Código IBGE do município', 
     'tipo': 'string', 
     'unidade': 'código IBGE', 
     'fonte': 'IBGE/Base dos Dados'},
    
    {'variavel': 'id_microrregiao', 
     'descricao': 'Código IBGE da microrregião', 
     'tipo': 'string', 
     'unidade': 'código IBGE', 
     'fonte': 'IBGE/Base dos Dados'},
    
    {'variavel': 'sigla_uf', 
     'descricao': 'Sigla da Unidade Federativa (estado)', 
     'tipo': 'string', 
     'unidade': 'sigla', 
     'fonte': 'IBGE/Base dos Dados'},
    
    # Variáveis de tratamento
    {'variavel': 'primeiro_ano_tratamento', 
     'descricao': 'Ano de instalação da primeira estação meteorológica automática (0 = sem estação)', 
     'tipo': 'int', 
     'unidade': 'ano', 
     'fonte': 'INMET/Base dos Dados'},
    
    {'variavel': 'tratado', 
     'descricao': 'Indicador se o município possui estação meteorológica (1 = sim, 0 = não)', 
     'tipo': 'int', 
     'unidade': 'binário', 
     'fonte': 'INMET/Base dos Dados'},
    
    {'variavel': 'pos_tratamento', 
     'descricao': 'Indicador se a observação é após instalação da estação (1 = sim, 0 = não)', 
     'tipo': 'int', 
     'unidade': 'binário', 
     'fonte': 'INMET/Base dos Dados'},
    
    # Variáveis socioeconômicas
    {'variavel': 'populacao_total', 
     'descricao': 'População total do município', 
     'tipo': 'float', 
     'unidade': 'habitantes', 
     'fonte': 'IBGE População/Base dos Dados'},
    
    {'variavel': 'pib_total', 
     'descricao': 'PIB total do município', 
     'tipo': 'float', 
     'unidade': 'reais', 
     'fonte': 'IBGE PIB Municipal/Base dos Dados'},
    
    {'variavel': 'pib_per_capita', 
     'descricao': 'PIB per capita do município (PIB/população)', 
     'tipo': 'float', 
     'unidade': 'reais/habitante', 
     'fonte': 'IBGE PIB Municipal/Base dos Dados'},
    
    {'variavel': 'pib_agropecuario', 
     'descricao': 'Valor adicionado bruto da agropecuária do município', 
     'tipo': 'float', 
     'unidade': 'reais', 
     'fonte': 'IBGE PIB Municipal/Base dos Dados'},
     
    # Variáveis de urbanização
    {'variavel': 'area_total_ha', 
     'descricao': 'Área total do município', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'IBGE/Base dos Dados'},
     
    {'variavel': 'pct_area_agropecuaria', 
     'descricao': 'Percentual da área do município dedicado à agropecuária', 
     'tipo': 'float', 
     'unidade': 'percentual', 
     'fonte': 'MapBiomas/Base dos Dados (calculado)'},
     
    {'variavel': 'pct_area_urbana', 
     'descricao': 'Percentual da área do município considerado urbano', 
     'tipo': 'float', 
     'unidade': 'percentual', 
     'fonte': 'MapBiomas/Base dos Dados (calculado)'},
     
    {'variavel': 'razao_agropecuaria_urbana', 
     'descricao': 'Razão entre área agropecuária e área urbana', 
     'tipo': 'float', 
     'unidade': 'razão', 
     'fonte': 'MapBiomas/Base dos Dados (calculado)'},
     
    {'variavel': 'municipio_urbano', 
     'descricao': 'Indicador se município é classificado como urbano (1 = urbano, 0 = rural)', 
     'tipo': 'int', 
     'unidade': 'binário', 
     'fonte': 'MapBiomas/Base dos Dados (calculado)'},
]

# Adicionar variáveis PAM para cada cultura irrigada
for cultura, cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.items():
    descricao_variaveis_list.extend([
        {'variavel': f'area_plantada_{cultura_id}', 
         'descricao': f'Área plantada de {cultura} segundo PAM', 
         'tipo': 'float', 
         'unidade': 'hectares', 
         'fonte': 'IBGE PAM/Base dos Dados'},
        
        {'variavel': f'area_colhida_{cultura_id}', 
         'descricao': f'Área colhida de {cultura} segundo PAM', 
         'tipo': 'float', 
         'unidade': 'hectares', 
         'fonte': 'IBGE PAM/Base dos Dados'},
        
        {'variavel': f'quantidade_{cultura_id}', 
         'descricao': f'Quantidade produzida de {cultura}', 
         'tipo': 'float', 
         'unidade': 'toneladas', 
         'fonte': 'IBGE PAM/Base dos Dados'},
        
        {'variavel': f'produtividade_{cultura_id}', 
         'descricao': f'Produtividade média de {cultura} (quantidade/área colhida)', 
         'tipo': 'float', 
         'unidade': 'kg/hectare', 
         'fonte': 'IBGE PAM/Base dos Dados'},
        
        {'variavel': f'valor_{cultura_id}', 
         'descricao': f'Valor da produção de {cultura}', 
         'tipo': 'float', 
         'unidade': 'mil reais', 
         'fonte': 'IBGE PAM/Base dos Dados'},
    ])

# Adicionar variáveis MapBiomas
descricao_variaveis_list.extend([
    {'variavel': 'area_agropecuaria', 
     'descricao': 'Área total dedicada à agropecuária (agricultura + pecuária)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
     
    {'variavel': 'area_agricultura', 
     'descricao': 'Área dedicada somente à agricultura', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_lavoura_temporaria', 
     'descricao': 'Área dedicada a lavouras temporárias', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_soja', 
     'descricao': 'Área dedicada ao cultivo de soja (MapBiomas)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_cana', 
     'descricao': 'Área dedicada ao cultivo de cana-de-açúcar (MapBiomas)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_outras_lavouras_temporarias', 
     'descricao': 'Área de outras lavouras temporárias (exceto soja e cana)', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_cafe', 
     'descricao': 'Área dedicada ao cultivo de café', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
    
    {'variavel': 'area_citricos', 
     'descricao': 'Área dedicada ao cultivo de cítricos', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'},
     
    {'variavel': 'area_area_urbana', 
     'descricao': 'Área urbana do município', 
     'tipo': 'float', 
     'unidade': 'hectares', 
     'fonte': 'MapBiomas/Base dos Dados'}
])

# Criar DataFrame
descricao_variaveis = pd.DataFrame(descricao_variaveis_list)

# Salvar dicionário de variáveis
dict_file = filepath.replace('.csv', '_dicionario_variaveis.csv')
descricao_variaveis.to_csv(dict_file, index=False, encoding='utf-8-sig')

print(f"\n📚 DICIONÁRIO DE VARIÁVEIS EXPORTADO!")
print(f"📁 Arquivo: {dict_file}")
print(f"📊 Total de variáveis descritas: {len(descricao_variaveis)}")

# Mostrar preview
print(f"\n🔍 Preview do dicionário:")
descricao_variaveis.head(15)



📚 DICIONÁRIO DE VARIÁVEIS EXPORTADO!
📁 Arquivo: data/dataset_mapbiomas_microrregiao_Cana-de-açúcar_2003-2021_20250912_dicionario_variaveis.csv
📊 Total de variáveis descritas: 22

🔍 Preview do dicionário:


,variavel,descricao,tipo,unidade,fonte
0,ano,Ano da observação,int,ano,Base dos Dados
1,id_microrregiao,Código do(a) microrregiao,string,código IBGE,IBGE/Base dos Dados
2,sigla_uf,Sigla da Unidade Federativa (estado),string,sigla,IBGE/Base dos Dados
3,primeiro_ano_tratamento,Ano de instalação da primeira estação meteorol...,int,ano,INMET/Base dos Dados
4,tratado,Indicador se a unidade possui estação meteorol...,int,binário,INMET/Base dos Dados
5,pos_tratamento,Indicador se a observação é após instalação da...,int,binário,INMET/Base dos Dados
6,pam_area_plantada,Área plantada de Cana-de-açúcar segundo PAM,float,hectares,IBGE PAM/Base dos Dados
7,pam_area_colhida,Área colhida de Cana-de-açúcar segundo PAM,float,hectares,IBGE PAM/Base dos Dados
8,pam_quantidade_produzida,Quantidade produzida de Cana-de-açúcar segundo...,float,toneladas,IBGE PAM/Base dos Dados
9,pam_produtividade,Produtividade média de Cana-de-açúcar (quantid...,float,kg/hectare,IBGE PAM/Base dos Dados


In [ ]:
# Criar versão em Markdown do dicionário
culturas_str = ', '.join(CULTURAS_IRRIGADAS_CORRIGIDO.keys())

markdown_content = f"""# Dicionário de Variáveis - Dataset MapBiomas com Culturas Irrigadas

**Dataset**: {filename}  
**Período**: {ANOS[0]}-{ANOS[-1]}  
**Granularidade**: Municipal (com id_microrregiao para agregação flexível)  
**Culturas irrigadas**: {culturas_str}  
**Data de geração**: {datetime.now().strftime('%d/%m/%Y')}

## Descrição das Variáveis

### 1. Variáveis de Identificação

- **ano**: Ano da observação (2003-2021)
- **id_municipio**: Código IBGE do município
- **id_microrregiao**: Código IBGE da microrregião (para agregação flexível)
- **sigla_uf**: Sigla da Unidade Federativa (estado)

### 2. Variáveis de Tratamento (Estações Meteorológicas)

- **primeiro_ano_tratamento**: Ano de instalação da primeira estação meteorológica automática no município. Valor 0 indica ausência de estação.
- **tratado**: Variável binária indicando se o município possui estação meteorológica (1 = sim, 0 = não)
- **pos_tratamento**: Variável binária indicando se a observação ocorre após a instalação da estação (1 = sim, 0 = não)

### 3. Variáveis Socioeconômicas

- **populacao_total**: População total do município (habitantes)
- **pib_total**: PIB total em reais correntes
- **pib_per_capita**: PIB per capita em reais/habitante
- **pib_agropecuario**: Valor adicionado bruto da agropecuária em reais

### 4. Variáveis de Urbanização

- **area_total_ha**: Área total do município em hectares
- **pct_area_agropecuaria**: Percentual da área do município dedicado à agropecuária (%)
- **pct_area_urbana**: Percentual da área do município considerado urbano (%)
- **razao_agropecuaria_urbana**: Razão entre área agropecuária e área urbana
- **municipio_urbano**: Classificação binária do município (1 = urbano, 0 = rural)
  - Município é considerado urbano se: % área urbana > 40% OU razão agropecuária/urbana < 1

### 5. Variáveis de Produção Agrícola (PAM) - Culturas Irrigadas

Para cada uma das 5 culturas com maior uso de irrigação no Brasil, são incluídas as seguintes variáveis:

"""

# Adicionar descrição para cada cultura
for cultura, cultura_id in CULTURAS_IRRIGADAS_CORRIGIDO.items():
    markdown_content += f"""
#### {cultura}
- **area_plantada_{cultura_id}**: Área plantada em hectares
- **area_colhida_{cultura_id}**: Área colhida em hectares
- **quantidade_{cultura_id}**: Quantidade produzida em toneladas
- **produtividade_{cultura_id}**: Produtividade média em kg/hectare (quantidade/área colhida)
- **valor_{cultura_id}**: Valor da produção em mil reais
"""

markdown_content += """
### 6. Variáveis de Uso da Terra (MapBiomas - Dados de Satélite)

- **area_agropecuaria**: Área total dedicada à agropecuária (agricultura + pecuária) (hectares)
- **area_agricultura**: Área dedicada somente à agricultura (hectares)
- **area_lavoura_temporaria**: Área dedicada a lavouras temporárias (hectares)
- **area_soja**: Área específica de cultivo de soja (hectares)
- **area_cana**: Área específica de cultivo de cana-de-açúcar (hectares)
- **area_outras_lavouras_temporarias**: Área de outras lavouras temporárias excluindo soja e cana (hectares)
- **area_cafe**: Área específica de cultivo de café (hectares)
- **area_citricos**: Área específica de cultivo de cítricos (hectares)
- **area_area_urbana**: Área urbana do município (hectares)

## Notas Metodológicas

1. **Granularidade Municipal**: O dataset mantém o id_municipio como menor unidade de identificação, permitindo análises detalhadas em nível municipal. O id_microrregiao é incluído para permitir agregações flexíveis.

2. **Culturas Irrigadas**: As 5 culturas selecionadas representam as maiores áreas irrigadas no Brasil:
   - Cana-de-açúcar: 1,7 milhão de hectares irrigados
   - Arroz: 1,1 milhão de hectares irrigados
   - Soja: 624 mil hectares irrigados
   - Milho: 559 mil hectares irrigados
   - Feijão: 195 mil hectares irrigados

3. **MapBiomas vs PAM**: Os dados do MapBiomas são baseados em imagens de satélite e tendem a ser mais precisos que os dados autodeclarados da PAM. Note que pode haver discrepâncias entre áreas reportadas pelo MapBiomas e pelo PAM.

4. **Produtividade**: A produtividade é calculada como quantidade produzida dividida pela área colhida. Valores zero indicam ausência de produção ou área colhida.

5. **Valores faltantes**: 
   - Zeros indicam ausência real do fenômeno (ex: sem produção)
   - NAs foram removidos apenas para população e PIB

6. **Tratamento**: O tratamento refere-se à presença de estações meteorológicas automáticas do INMET, que fornecem dados climáticos mais precisos e frequentes.

7. **Classificação de Urbanização**: A classificação de municípios como urbanos ou rurais é baseada em dois critérios:
   - Área urbana superior a 40% da área total do município, OU
   - Razão entre área agropecuária e área urbana menor que 1 (área urbana > área agropecuária)
   - Esta classificação usa dados de uso da terra do MapBiomas, que são baseados em imagens de satélite

## Fontes dos Dados

- **IBGE**: População, PIB Municipal, Produção Agrícola Municipal (PAM)
- **INMET**: Estações meteorológicas
- **MapBiomas**: Cobertura e uso da terra
- **Base dos Dados**: Plataforma de acesso integrado aos dados

---
*Documento gerado automaticamente pelo notebook de extração de dados*
"""

# Salvar arquivo Markdown
md_file = filepath.replace('.csv', '_dicionario_variaveis.md')
with open(md_file, 'w', encoding='utf-8') as f:
    f.write(markdown_content)

print(f"\n📝 DICIONÁRIO EM MARKDOWN EXPORTADO!")
print(f"📁 Arquivo: {md_file}")
print(f"\n✅ Arquivos de documentação criados com sucesso!")
print("   - CSV com tabela de variáveis")
print("   - Markdown com descrição detalhada")
print("   - JSON com metadados do dataset")
